# Linear Regression 2

Given a csv data file, format it into MLlib "feature" & "label" and uild a regression model that will predict the customer's yearly spend on the company's product.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [7]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [10]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [11]:
# data format into ("label","features") - only the last 5 numerical features

# Import VectorAssembler and Vectors

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
# specify all the features you wonna use

assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [14]:
output = assembler.transform(data)

In [15]:
# combination of all 4 columns
output.select("features").show(5)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
+--------------------+
only showing top 5 rows



In [16]:
output.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [17]:
# features & label (Yearly Amount Spent)
final_data = output.select("features",'Yearly Amount Spent')

In [18]:
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [19]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                351|
|   mean|  500.8026077078109|
| stddev|  80.91555321095288|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                149|
|   mean|  495.8074082137835|
| stddev|  75.55882607983445|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [24]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} \n \n Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.256655565942417,38.816354938366906,0.275069850669323,61.50290835699616] 
 
 Intercept: -1030.6832069701736


In [25]:
test_results = lrModel.evaluate(test_data)

In [26]:
# Interesting results....
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -6.835881442121774|
|  -5.764660022803525|
|  -7.398920487533417|
|  -8.497404169740093|
|   8.649445033411894|
|  -2.109029904783597|
|  1.7882219102910994|
|  2.8329961358016362|
|-0.15681911976935226|
|  -5.011077286121235|
|  -8.212065488176108|
|  -15.28830818466463|
| -2.8369314487900965|
| -1.8188639275857668|
| -27.272503032622467|
|   -7.24969937034615|
|  1.6067620284175632|
|  -5.325305221720839|
| -11.993591437012356|
|  -4.593688578286674|
+--------------------+
only showing top 20 rows



In [27]:
print("MSE Score: {}".format(test_results.meanSquaredError))
print()
print("RMSE Score: {}".format(test_results.rootMeanSquaredError))
print()
print("R2 Score: {}".format(test_results.r2))
print()
print("MAE Score: {}".format(test_results.meanAbsoluteError))
print()
print("Explained Variance Score: {}".format(test_results.explainedVariance))

MSE Score: 90.98553620601375

RMSE Score: 9.538633875247218

R2 Score: 0.983955488158506

MAE Score: 7.476286689361867

Explained Variance Score: 5492.710232711447


# Compare with unlabelled data - data with features but no label (predict)

In [32]:
unlabeled_data = test_data.select('features')

In [33]:
predictions = lrModel.transform(unlabeled_data)

In [34]:
predictions.show(5)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 289.3071271620363|
|[30.8364326747734...|473.26656044979313|
|[31.0613251567161...|  494.954378545435|
|[31.1280900496166...| 565.7500909167948|
|[31.1695067987115...| 418.7070857688809|
+--------------------+------------------+
only showing top 5 rows



In [35]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.538633875247218
MSE: 90.98553620601375
